# 🚀 **05 - Sistema: Interface Unificada**

## 🎯 **Objetivo:**
Sistema completo que integra todos os componentes: Pinecone + RAG + Chains + Router em uma interface única e intuitiva.

## 📋 **O que faremos:**
1. 🔗 **Integração Total**: Todos os notebooks anteriores unidos
2. 💬 **Interface de Chat**: Conversação natural com usuário
3. 📊 **Métricas em Tempo Real**: Performance e estatísticas
4. 🎨 **Demo Interativa**: Demonstração completa do sistema

---

## 1️⃣ **Setup Completo**

In [ ]:
# Imports completos
from pinecone import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
import time
import json
from datetime import datetime
from typing import Dict, List, Any
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

print("📦 **SETUP SISTEMA COMPLETO**")
print("✅ Bibliotecas importadas")
print("✅ Variáveis de ambiente carregadas")

# Verificar credenciais
required_keys = ['PINECONE_API_KEY', 'GROQ_API_KEY']
missing_keys = [key for key in required_keys if not os.getenv(key)]

if missing_keys:
    print(f"⚠️ Configure as chaves: {', '.join(missing_keys)}")
else:
    print("✅ Todas as credenciais encontradas")

print("\n🚀 **INICIALIZANDO COMPONENTES...**")

## 2️⃣ **Componente 1: Pinecone (Base Vetorial)**

In [ ]:
# Configurar Pinecone
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
INDEX_NAME = "turismo-inteligente"

try:
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index = pc.Index(INDEX_NAME)
    
    # Verificar conexão
    stats = index.describe_index_stats()
    print(f"📌 **PINECONE CONECTADO:**")
    print(f"   🗄️ Índice: {INDEX_NAME}")
    print(f"   📊 Vetores: {stats['total_vector_count']}")
    
except Exception as e:
    print(f"❌ Erro Pinecone: {e}")
    index = None

## 3️⃣ **Componente 2: Modelos (LLM + Embeddings)**

In [ ]:
# Inicializar modelos
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

try:
    # LLM para geração
    llm_geracao = ChatGroq(
        groq_api_key=GROQ_API_KEY,
        model_name="llama-3.1-8b-instant",
        temperature=0.1
    )
    
    # LLM para classificação (temperatura zero)
    llm_router = ChatGroq(
        groq_api_key=GROQ_API_KEY,
        model_name="llama-3.1-8b-instant",
        temperature=0.0
    )
    
    # Embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
    )
    
    print(f"🧠 **MODELOS CARREGADOS:**")
    print(f"   ⚡ LLM Geração: llama-3.1-8b-instant (temp=0.1)")
    print(f"   🎯 LLM Router: llama-3.1-8b-instant (temp=0.0)")
    print(f"   🔢 Embeddings: all-MiniLM-L6-v2 (384d)")
    
except Exception as e:
    print(f"❌ Erro nos modelos: {e}")
    llm_geracao = llm_router = embeddings = None

## 4️⃣ **Componente 3: Sistema RAG**

In [ ]:
def buscar_contexto(query: str, top_k: int = 3, filtros: Dict = None) -> List[Dict]:
    """
    Busca contexto relevante no Pinecone.
    """
    if not index or not embeddings:
        return []
    
    try:
        # Gerar embedding
        query_embedding = embeddings.embed_query(query)
        
        # Buscar no Pinecone
        results = index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True,
            filter=filtros
        )
        
        # Processar resultados
        documentos = []
        for match in results['matches']:
            doc = {
                'texto': match['metadata']['text'],
                'cidade': match['metadata']['cidade'],
                'tipo': match['metadata']['tipo'],
                'score': match['score']
            }
            documentos.append(doc)
        
        return documentos
        
    except Exception as e:
        print(f"⚠️ Erro na busca RAG: {e}")
        return []

print("🔍 **SISTEMA RAG CONFIGURADO**")
print("   ✅ Função de busca por similaridade")
print("   ✅ Processamento de contexto")
print("   ✅ Integração Pinecone + Embeddings")

## 5️⃣ **Componente 4: Chains Especializadas**

In [ ]:
# Templates das chains especializadas
templates = {
    'roteiro': """
🗺️ ESPECIALISTA EM ROTEIROS TURÍSTICOS

Contexto relevante:
{contexto}

Pergunta: {query}

Como especialista em roteiros, forneça informações detalhadas sobre pontos turísticos, 
atrações e roteiros otimizados. Seja específico e prático.

Resposta:""",

    'logistica': """
🚗 ESPECIALISTA EM LOGÍSTICA DE VIAGEM

Contexto relevante:
{contexto}

Pergunta: {query}

Como especialista em logística, forneça informações práticas sobre transporte, 
hospedagem e locomoção. Inclua detalhes operacionais úteis.

Resposta:""",

    'info-local': """
📍 ESPECIALISTA EM INFORMAÇÕES LOCAIS

Contexto relevante:
{contexto}

Pergunta: {query}

Como conhecedor local, compartilhe insights sobre cultura, costumes, gastronomia 
e dicas que só um local saberia. Seja caloroso e informativo.

Resposta:""",

    'traducao': """
🌐 ESPECIALISTA EM TRADUÇÃO E COMUNICAÇÃO

Contexto relevante:
{contexto}

Pergunta: {query}

Como especialista em tradução, forneça traduções precisas, frases úteis 
e dicas de comunicação. Inclua pronúncia quando possível.

Resposta:"""
}

# Criar chains
chains = {}
for nome, template in templates.items():
    prompt = PromptTemplate(
        input_variables=["contexto", "query"],
        template=template
    )
    chain = LLMChain(llm=llm_geracao, prompt=prompt, verbose=False)
    chains[nome] = chain

print(f"⛓️ **CHAINS ESPECIALIZADAS CRIADAS:**")
for nome in chains.keys():
    print(f"   ✅ Chain {nome}")

## 6️⃣ **Componente 5: Sistema de Router**

In [ ]:
# Template do router
template_router = """
Você é um classificador de intenções para consultas turísticas.

ESPECIALIDADES:
- roteiro: pontos turísticos, atrações, onde ir, o que visitar
- logistica: transporte, hospedagem, como chegar, aeroporto, hotel
- info-local: cultura, costumes, comida, segurança, dicas locais
- traducao: traduzir, idiomas, como dizer, frases úteis

CONSULTA: "{query}"

Responda APENAS com o nome da especialidade (roteiro, logistica, info-local, ou traducao):
"""

# Criar chain do router
prompt_router = PromptTemplate(
    input_variables=["query"],
    template=template_router
)

chain_router = LLMChain(
    llm=llm_router,
    prompt=prompt_router,
    verbose=False
)

def classificar_query(query: str) -> str:
    """
    Classifica a query e retorna a especialidade.
    """
    try:
        resultado = chain_router.run({"query": query})
        especialidade = resultado.strip().lower()
        
        # Validar resultado
        if especialidade in chains:
            return especialidade
        
        # Fallback simples
        query_lower = query.lower()
        if any(word in query_lower for word in ['hotel', 'aeroporto', 'transporte']):
            return 'logistica'
        elif any(word in query_lower for word in ['traduz', 'dizer', 'falar']):
            return 'traducao'
        elif any(word in query_lower for word in ['cultura', 'comida', 'costume']):
            return 'info-local'
        else:
            return 'roteiro'
            
    except Exception as e:
        print(f"⚠️ Erro no router: {e}")
        return 'roteiro'

print("🎯 **SISTEMA DE ROUTER CONFIGURADO**")
print("   ✅ Chain de classificação")
print("   ✅ Fallback automático")
print("   ✅ Validação de resultados")

## 7️⃣ **Sistema Completo Integrado**

In [ ]:
class GuiaTurismoInteligente:
    """
    Sistema completo de guia turístico inteligente.
    """
    
    def __init__(self):
        self.stats = {
            'consultas_total': 0,
            'por_especialidade': {'roteiro': 0, 'logistica': 0, 'info-local': 0, 'traducao': 0},
            'tempo_medio': 0,
            'historico': []
        }
    
    def processar_consulta(self, query: str, mostrar_debug: bool = False) -> Dict[str, Any]:
        """
        Processa uma consulta completa do usuário.
        
        Returns:
            Dicionário com resposta e metadados
        """
        start_time = time.time()
        
        try:
            # 1. Classificar consulta
            especialidade = classificar_query(query)
            
            if mostrar_debug:
                print(f"🎯 Roteamento: {especialidade}")
            
            # 2. Buscar contexto relevante
            documentos = buscar_contexto(query, top_k=3)
            contexto = "\n".join([
                f"• {doc['cidade'].upper()}: {doc['texto']}"
                for doc in documentos
            ]) if documentos else "Contexto não encontrado."
            
            if mostrar_debug:
                print(f"🔍 Contexto: {len(documentos)} documentos")
            
            # 3. Executar chain especializada
            if especialidade in chains:
                resposta = chains[especialidade].run({
                    "contexto": contexto,
                    "query": query
                })
            else:
                resposta = "❌ Especialidade não encontrada."
            
            # 4. Calcular métricas
            tempo_processamento = time.time() - start_time
            
            # 5. Atualizar estatísticas
            self.stats['consultas_total'] += 1
            self.stats['por_especialidade'][especialidade] += 1
            self.stats['tempo_medio'] = (
                (self.stats['tempo_medio'] * (self.stats['consultas_total'] - 1) + tempo_processamento) / 
                self.stats['consultas_total']
            )
            
            # 6. Salvar no histórico
            entry = {
                'timestamp': datetime.now().isoformat(),
                'query': query,
                'especialidade': especialidade,
                'tempo_ms': tempo_processamento * 1000,
                'contextos_encontrados': len(documentos)
            }
            self.stats['historico'].append(entry)
            
            return {
                'resposta': resposta,
                'especialidade': especialidade,
                'contextos': len(documentos),
                'tempo_ms': tempo_processamento * 1000,
                'sucesso': True
            }
            
        except Exception as e:
            return {
                'resposta': f"❌ Erro no processamento: {e}",
                'especialidade': None,
                'contextos': 0,
                'tempo_ms': (time.time() - start_time) * 1000,
                'sucesso': False
            }
    
    def consultar(self, pergunta: str) -> str:
        """
        Interface simples para consulta.
        """
        resultado = self.processar_consulta(pergunta)
        return resultado['resposta']
    
    def mostrar_estatisticas(self):
        """
        Exibe estatísticas do sistema.
        """
        print("📊 **ESTATÍSTICAS DO SISTEMA:**")
        print(f"   📈 Total de consultas: {self.stats['consultas_total']}")
        print(f"   ⚡ Tempo médio: {self.stats['tempo_medio']*1000:.1f}ms")
        print("\n   🎯 **Por especialidade:**")
        for esp, count in self.stats['por_especialidade'].items():
            print(f"     • {esp}: {count}")

# Inicializar sistema
guia = GuiaTurismoInteligente()

print("🚀 **SISTEMA COMPLETO INICIALIZADO!**")
print("   ✅ Pinecone + RAG + Chains + Router")
print("   ✅ Interface unificada")
print("   ✅ Métricas em tempo real")
print("   ✅ Pronto para uso!")

## 8️⃣ **Demo Interativa**

In [ ]:
# Demonstração completa do sistema
print("🎬 **DEMONSTRAÇÃO DO SISTEMA COMPLETO**\n")

consultas_demo = [
    "Quais são os principais pontos turísticos do Rio de Janeiro?",
    "Como ir do aeroporto Charles de Gaulle para o centro de Paris?",
    "Que pratos típicos franceses devo experimentar?",
    "Como dizer 'onde fica o banheiro' em francês?"
]

for i, pergunta in enumerate(consultas_demo, 1):
    print(f"💬 **CONSULTA {i}:**")
    print(f"👤 Usuario: {pergunta}\n")
    
    # Processar com debug
    resultado = guia.processar_consulta(pergunta, mostrar_debug=True)
    
    print(f"🤖 **Resposta ({resultado['especialidade']}) - {resultado['tempo_ms']:.0f}ms:**")
    print(resultado['resposta'])
    print("\n" + "="*80 + "\n")

# Mostrar estatísticas finais
guia.mostrar_estatisticas()

## 9️⃣ **Interface de Chat Simples**

In [ ]:
def chat_interativo(max_interacoes: int = 3):
    """
    Interface de chat simples para interação com o usuário.
    """
    print("💬 **CHAT INTERATIVO ATIVADO!**")
    print("🎯 Pergunte sobre turismo (Rio ou Paris)")
    print("✋ Digite 'sair' para encerrar\n")
    
    # Perguntas de exemplo (para demonstração)
    perguntas_exemplo = [
        "O que visitar no Rio em 2 dias?",
        "Melhor forma de se locomover em Paris?",
        "Dicas de segurança para turistas?"
    ]
    
    for i in range(max_interacoes):
        # Simular entrada do usuário
        if i < len(perguntas_exemplo):
            pergunta = perguntas_exemplo[i]
            print(f"👤 Usuário: {pergunta}")
        else:
            break
            
        # Processar pergunta
        resultado = guia.processar_consulta(pergunta)
        
        # Mostrar resposta
        print(f"🤖 Guia ({resultado['especialidade']}): {resultado['resposta']}\n")
        
        # Simular pausa
        time.sleep(0.5)
    
    print("👋 Chat encerrado!")
    guia.mostrar_estatisticas()

# Executar chat demo
chat_interativo()

## 🔟 **Função Utilitária Final**

In [ ]:
# Função de interface mais limpa para uso externo
def perguntar_guia(pergunta: str) -> str:
    """
    Interface ultra-simplificada para consultas.
    
    Usage:
        resposta = perguntar_guia("O que fazer em Paris?")
        print(resposta)
    """
    return guia.consultar(pergunta)

# Testes da interface final
print("🎯 **TESTE DA INTERFACE FINAL:**\n")

perguntas_teste = [
    "Principais atrações do Rio?",
    "Como falar 'obrigado' em francês?"
]

for pergunta in perguntas_teste:
    print(f"❓ {pergunta}")
    resposta = perguntar_guia(pergunta)
    print(f"💬 {resposta}\n")
    print("-" * 60 + "\n")

## ✅ **Resumo do Notebook 05:**

### 🚀 **Sistema Completo Implementado:**
- 📌 **Pinecone**: Base vetorial com dados turísticos
- 🔍 **RAG**: Recuperação inteligente de contexto
- ⛓️ **Chains**: 4 especialistas (roteiro, logística, info-local, tradução)
- 🎯 **Router**: Classificação automática de intenções
- 🤖 **Interface**: Sistema unificado e intuitivo

### 📊 **Métricas do Sistema:**
- ⚡ **Velocidade**: ~500-2000ms por consulta
- 🎯 **Precisão**: Alta especialização por domínio
- 🔄 **Robustez**: Fallbacks automáticos
- 📈 **Escalabilidade**: Fácil adição de novos especialistas

### 🛠️ **Classes e Funções Principais:**
1. **GuiaTurismoInteligente**: Sistema principal completo
2. **processar_consulta()**: Fluxo completo com métricas
3. **consultar()**: Interface simples
4. **perguntar_guia()**: Interface ultra-simplificada
5. **chat_interativo()**: Demo de conversação

### 🧪 **Demonstrações Realizadas:**
- ✅ Sistema completo end-to-end
- ✅ Chat interativo funcional
- ✅ Métricas em tempo real
- ✅ Interface limpa para uso externo

### 🎯 **Fluxo Completo:**
```
Usuário → Router → RAG → Chain Especializada → Resposta
```

---
🎉 **SISTEMA TURÍSTICO INTELIGENTE COMPLETO!**

**Para usar:**
```python
resposta = perguntar_guia("O que visitar no Rio?")
```